In [ ]:
!pip install --quiet openai gspread pandas

In [4]:
import openai
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import pandas as pd

sheet_url = "https://docs.google.com/spreadsheets/d/19VnVlNtUBOjdcYtKoTuwUKFEsJLmh0XFH8cYfdfKRWI/export?format=csv&gid=2121959045"

students = pd.read_csv(sheet_url)

students.head()

,Timestamp,Student Name,Age,Category,Family Income,Marks(%),Course/Class,State,Gender,Eligible Scholarships,Not Eligible Scholarships


In [3]:
results = []

for i, student in students.iterrows():
    prompt = f"""
You are an AI-based Scholarship Eligibility Evaluator.

Your task is to evaluate scholarship eligibility using
GENERAL SCHOLARSHIP PATTERNS, not fixed or specific scheme names.

Student details:
Name: {student['Student Name']}
Age: {student['Age']}
Gender: {student['Gender']}
Category: {student['Category']}
Family Income: {student['Family Income']}
Marks: {student['Marks (%)']}
Course/Class: {student['Course/Class']}
State: {student['State']}

Scholarship Evaluation Guidelines:
• Merit-based scholarships depend on high academic marks
• Income-based welfare scholarships depend on low family income
• Category-based scholarships depend on SC/ST/OBC status
• Gender-based scholarships depend on Female / Girl Child eligibility
• State-level scholarships depend on domicile/state
• Course/Class-based scholarships depend on education level

Instructions:
• Identify which TYPES of scholarships the student is eligible for
• Identify which TYPES of scholarships the student is NOT eligible for
• Do NOT invent real government scheme names
• Use only general categories like:
  "Merit-based Scholarship",
  "Income-based Welfare Scholarship",
  "Category-based Scholarship",
  "Women Welfare Scholarship",
  "State-level Scholarship"

Return ONLY valid JSON in this exact format:
{
  "Eligible_Scholarships": [],
  "Not_Eligible_Scholarships": []
}
"""

    # Call OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

    # Get AI response
    text = response['choices'][0]['message']['content']

    # Parse JSON safely
    try:
        data = json.loads(text)
    except:
        data = {
            "Eligible_Scholarships": [],
            "Not_Eligible_Scholarships": []
        }

    # Convert list output to text
    eligible = ", ".join(data["Eligible_Scholarships"])
    not_eligible = ", ".join(data["Not_Eligible_Scholarships"])

    # Update Google Sheet
    # Column 9 → Eligible Scholarships
    # Column 10 → Not Eligible Scholarships
    sheet.update_cell(i + 2, 9, eligible)
    sheet.update_cell(i + 2, 10, not_eligible)

    results.append(text)

# Optional: print results in Colab
for r in results:
    print(r)

print("✅ Scholarship eligibility updated in Google Sheet successfully!")


✅ Scholarship eligibility updated in Google Sheet successfully!


In [ ]:
# Save AI results to a local JSON file
with open("scholarship_results.json", "w") as f:
    for r in results:
        f.write(r + "\n")

print("✅ Results saved as scholarship_results.json")

✅ Results saved as scholarship_results.json
